2nd Question

Part (**a**)

In [61]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os

from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object
print(device)

cuda:0


In [63]:
# Load the pre-trained model
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')   #load resnet18 model
num_features = model.fc.in_features     #extract fc layers features 
model = torch.nn.Sequential(*list(model.children())[:-1])
model = model.to(device) #move model to device
criterion = nn.CrossEntropyLoss()  #(set loss function)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [64]:
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [65]:
transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

In [66]:
train_dir = "/content/drive/MyDrive/AIP_data/train"
test_dir = "/content/drive/MyDrive/AIP_data/test"
batchSize = 532

In [67]:
train_dataset = datasets.ImageFolder(train_dir, transform)
test_dataset = datasets.ImageFolder(test_dir, transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batchSize, shuffle=False, num_workers=2)

In [68]:
print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))
class_names = train_dataset.classes
print('Class names:', class_names)

Train dataset size: 532
Test dataset size: 120
Class names: ['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']


In [69]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [70]:
# Saving train features
model.eval()
train_features = np.zeros(shape=(532, 512, 1, 1))
train_labels = np.zeros(shape=(532,1))
batch_size = 4
with torch.no_grad():
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        train_features[i * batch_size: (i + 1) * batch_size] = outputs.cpu().data.numpy().argmax()
        train_labels[i * batch_size: (i + 1) * batch_size] = labels.cpu().data.numpy().argmax()

In [71]:
# Saving test features
model.eval()
test_features = np.zeros(shape=(120, 512, 1, 1))
test_labels = np.zeros(shape=(120,1))
batch_size = 4
with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        test_features[i * batch_size: (i + 1) * batch_size] = outputs.cpu().data.numpy().argmax()
        test_labels[i * batch_size: (i + 1) * batch_size] = labels.cpu().data.numpy().argmax()

In [72]:
train_features = GlobalAveragePooling2D()(train_features).numpy()
test_features = GlobalAveragePooling2D()(test_features).numpy()

In [73]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_features, train_labels)

pred_labels = knn.predict(test_features)

# Evaluate the accuracy of the k-NN classifier
accuracy = accuracy_score(test_labels, np.ravel(pred_labels,order='C'))
print("Accuracy:", accuracy)

Accuracy: 0.9666666666666667


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Part (**b**)

In [74]:
# Load the pre-trained model
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')   #load resnet18 model
num_features = model.fc.in_features     #extract fc layers features 
model.fc = nn.Linear(num_features, 6) #(num_of_class == 6)
model = model.to(device) #move model to device
criterion = nn.CrossEntropyLoss()  #(set loss function)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [75]:
num_epochs = 25   #(set no of epochs)
start_time = time.time() #(for showing time)
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.cpu()
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))
    
    """ Testing Phase """
model.eval()
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        outputs = outputs.cpu()
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset) * 100.
    print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 0 running
[Train #0] Loss: 2.0184 Acc: 12.2180% Time: 5.4965s
Epoch 1 running
[Train #1] Loss: 1.6922 Acc: 28.5714% Time: 10.9989s
Epoch 2 running
[Train #2] Loss: 1.4249 Acc: 53.7594% Time: 16.3724s
Epoch 3 running
[Train #3] Loss: 1.1819 Acc: 69.7368% Time: 21.8138s
Epoch 4 running
[Train #4] Loss: 0.9007 Acc: 77.2556% Time: 27.1871s
Epoch 5 running
[Train #5] Loss: 0.6153 Acc: 86.4662% Time: 32.6960s
Epoch 6 running
[Train #6] Loss: 0.3974 Acc: 94.9248% Time: 38.4307s
Epoch 7 running
[Train #7] Loss: 0.2762 Acc: 97.5564% Time: 43.7648s
Epoch 8 running
[Train #8] Loss: 0.2155 Acc: 97.3684% Time: 50.0935s
Epoch 9 running
[Train #9] Loss: 0.1757 Acc: 97.7444% Time: 55.5115s
Epoch 10 running
[Train #10] Loss: 0.1404 Acc: 98.1203% Time: 60.9330s
Epoch 11 running
[Train #11] Loss: 0.1075 Acc: 98.4962% Time: 66.3954s
Epoch 12 running
[Train #12] Loss: 0.0797 Acc: 98.8722% Time: 72.2273s
Epoch 13 running
[Train #13] Loss: 0.0586 Acc: 99.4361% Time: 77.6140s
Epoch 14 running
[Train #14

In [76]:
save_path = 'image-classifier_resnet_18_final_60_last_tr_epochs.pth'
torch.save(model.state_dict(), save_path)

In [77]:
model.load_state_dict(torch.load("image-classifier_resnet_18_final_60_last_tr_epochs.pth"))

<All keys matched successfully>

In [78]:
# Testing model for train data
model.eval()
start_time = time.time()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
train_features = np.zeros(shape=(528, 512, 1, 1))
train_labels = np.zeros(shape=(528,1))
batch_size = 1
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        output = feature_extractor(inputs)
        train_features[i * batch_size: (i + 1) * batch_size] = output.cpu().data.numpy().argmax()
        train_labels[i * batch_size: (i + 1) * batch_size] = labels.cpu().data.numpy().argmax()
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.
          format( epoch_loss, epoch_acc, time.time() - start_time))

[Train #] Loss: 0.0024 Acc: 100.0000% Time: 5.8952s


In [79]:
# Testing model for test data
model.eval()
start_time = time.time()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
test_features = np.zeros(shape=(120, 512, 1, 1))
test_labels = np.zeros(shape=(120,1))
batch_size = 1
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
    for i, (inputs, labels) in enumerate(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        output = feature_extractor(inputs)
        test_features[i * batch_size: (i + 1) * batch_size] = outputs.cpu().data.numpy().argmax()
        test_labels[i * batch_size: (i + 1) * batch_size] = labels.cpu().data.numpy().argmax()
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset) * 100.
    print('[Test #] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.
          format( epoch_loss, epoch_acc, time.time() - start_time))

[Test #] Loss: 0.1359 Acc: 95.0000% Time: 1.2469s


Part (**c**)

In [80]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [81]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel,self).__init__()
    self.layer1 = nn.Flatten()
    self.fc1 = nn.Linear(224*224*3,6)

  def forward(self,x):
    x = self.layer1(x)
    x = self.fc1(x)
    return x

# Initialize the network
model = MyModel()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Load the training data
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
train_dir = "/content/drive/MyDrive/AIP_data/train"
test_dir = "/content/drive/MyDrive/AIP_data/test"
batchSize = 1
train_dataset = datasets.ImageFolder(train_dir, transform)
test_dataset = datasets.ImageFolder(test_dir, transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batchSize, shuffle=False, num_workers=2)
print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))
class_names = train_dataset.classes
print('Class names:', class_names)

# Train the network
for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_dataloader, 0):
          
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i == 0:    # print once for each epoch
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0


Train dataset size: 532
Test dataset size: 120
Class names: ['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']
[1,     1] loss: 0.002
[2,     1] loss: 0.000
[3,     1] loss: 1.126
[4,     1] loss: 0.000
[5,     1] loss: 0.000


In [82]:
save_path = 'image-classifier_cnn_final_10_last_tr_epochs.pth'
torch.save(model.state_dict(), save_path)

In [83]:
model.load_state_dict(torch.load("image-classifier_cnn_final_10_last_tr_epochs.pth"))

<All keys matched successfully>

In [84]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total  # accuracy
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 30.00%
